# Setup: libraries import and data loading

In [5]:
import numpy as np
import pandas as pd
import sklearn

In [7]:
df = pd.read_excel("datos_ine.xlsx")

# Dataset cleaning

We are asked to 

> Remove the observations where the domestic sales or exports are either negative or missing (they appear as “.”) or where the total sales are null or negative.

But from running `df.dtypes` we can see that all columns have been correctly parsed as integres, meaning that there are no strings like "." in the data:

In [19]:
df.dtypes

Obs       int64
year      int64
nui       int64
ciiu3     int64
region    int64
forpro    int64
fabval    int64
expval    int64
va        int64
emptot    int64
rempag    int64
vstk      int64
dtype: object

We can even see that there are no missing (N/A) values in the dataset, using the handy `dropna()` function:

In [38]:
len(df)

31316

In [24]:
df2 = df.dropna()
assert(len(df2)==len(df))

Furthermore, there is no column describing "domestic sales": we only have "Exports" (`EXPVAL`) and "Total Sales"(`FABVAL`). Assuming total sales equal domestic sales + exports, we can compute the missing column. We can also use more expressive names:

In [25]:
df["exports"] = df["expval"]
df["total_sales"] = df["fabval"]
df["domestic_sales"] = df["total_sales"] - df["exports"]

Even if we have data points for all of the observations, we can still have some negative values, which do not make sense in this context. We find only one "dirty" observation:

In [40]:
errors = df[(df.total_sales < 0) | (df.exports < 0) | (df.domestic_sales < 0)]
errors

,Obs,year,nui,ciiu3,region,forpro,fabval,expval,va,emptot,rempag,vstk,exports,total_sales,domestic_sales
790,791,2001,10932,2925,8,1,0,1,25504,21,85073,1700000,1,0,-1


In [41]:
df = df[(df.total_sales >= 0) & (df.exports >= 0) & (df.domestic_sales >= 0)]

In [43]:
len(df)

31315

Looking ad data from nui=10004 we can see that even if a company has only one year of activity, there will be observations for the whole period [2001, 2006], to keep the panel balanced.

In [47]:
df[df.nui==10004]

,Obs,year,nui,ciiu3,region,forpro,fabval,expval,va,emptot,rempag,vstk,exports,total_sales,domestic_sales
4,5,2001,10004,3592,1,1,1355082,0,1967145,60,502000,5600000,0,1355082,1355082
4950,4951,2002,10004,3592,1,1,0,0,1695807,54,556135,2500000,0,0,0
10216,10217,2003,10004,3592,1,1,0,0,2251406,73,586546,3200000,0,0,0
15442,15443,2004,10004,3592,1,1,0,0,1365630,124,924612,1400000,0,0,0
20873,20874,2005,10004,3511,1,1,0,0,1162824,123,1037220,1400000,0,0,0
26215,26216,2006,10004,3511,1,1,0,0,1250395,119,990909,6400000,0,0,0


Therefore, and since we do not have a way to find missing data for those companies who do not have exactly 6 observations, we will need to exclude any company that does not have exactly 6 observations from the bataset in order to keep the panel balanced.

In [60]:
df["nui"].nunique()

7797

In [71]:
nuis_obs = df[["Obs","nui"]].groupby("nui").count()
nuis = nuis_obs.index.values
balanced_nuis = nuis_obs[nuis_obs["Obs"] == 6].index.values

In [76]:
print("There are {} companies in the panel, but only {} ({:3.1f}%) with exactly six observations.".format(len(nuis), len(balanced_nuis), 100.0 * len(balanced_nuis) / len(nuis)))

There are 7797 companies in the panel, but only 2825 (36.2%) with exactly six observations.


Having to discard two thirds of the companies due to aparrently missing data seems a bit drastic, and therefore in a real setting we might try to investigate the issue further to see what can be done.

In [95]:
balanced_df = df[df["nui"].isin(balanced_nuis)]
assert(len(balanced_df) == len(balanced_nuis) * 6)

In [109]:
pivoted_df = balanced_df.pivot(index='nui', columns='year', values='exports').reset_index()[["nui", 2001, 2006]]

In [112]:
def classify_nui(row):
    if (row[2001] > 0) and (row[2006] > 0):
        return "Cont. Exporter"
    elif (row[2001] > 0) and (row[2006] == 0):
        return "Stopper"
    elif (row[2001] == 0) and (row[2006] > 0):
        return "Starter"
    elif (row[2001] ==0) and (row[2006] == 0):
        return "Cont. Non-exporter"
    else:
        return "ERROR"

In [113]:
pivoted_df["category"] = pivoted_df.apply(lambda row: classify_nui(row),axis=1)